In [ ]:

!pip install telethon nest_asyncio emoji pandas numpy==2.* --quiet

import re
import asyncio
import nest_asyncio
import pandas as pd
import emoji
from telethon import TelegramClient


api_id = ***
api_hash = ***
phone = ***


channel_username = "kunuzen"
output_file = "Uzbek_Latin_400.txt"
limit_messages = 10000
target_sentences = 400


def remove_emojis(text: str) -> str:
    return emoji.replace_emoji(text, replace='')


def clean_message(text: str) -> str:
    text = remove_emojis(text)
    
    text = re.sub(r"http\S+|www\S+|t\.me\S+|\@\S+|\.uz|\.kr", "", text, flags=re.IGNORECASE)
    text = re.sub(r"instagram|youtube|facebook|telegram|tiktok|x\.com|twitter", "", text, flags=re.IGNORECASE)
    
    text = re.sub(r"[^A-Za-z0-9\s\.,!?'\-]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


async def fetch_messages():
    async with TelegramClient("uzbek_latin_session", api_id, api_hash) as client:
        await client.start(phone)
        print("✅ Signed in successfully!")

        print(f"📥 Fetching messages from @{channel_username} ...")
        clean_texts = []

        async for msg in client.iter_messages(channel_username, limit=limit_messages):
            if not msg.text:
                continue
            cleaned = clean_message(msg.text)
            if not cleaned:
                continue
            if len(cleaned.split()) < 4:
                continue   
            clean_texts.append(cleaned)

            if len(clean_texts) >= target_sentences:
                break

        print(f"💾 Saved {len(clean_texts)} cleaned sentences to {output_file}")
        with open(output_file, "w", encoding="utf-8") as f:
            f.write("\n".join(clean_texts))


nest_asyncio.apply()
await fetch_messages()

